Imports

In [3]:
import numpy as np
import pandas as pd
import sklearn as sklearn
from sklearn.ensemble import HistGradientBoostingClassifier
print('Now using sklearn version '+sklearn.__version__)
print('packages are ready!')

Now using sklearn version 1.0.2
packages are ready!


Loading data

In [4]:
print('Loading data...')
"""we are also importing training and testing data from competition because they might become handy"""

test=pd.read_csv('/Users/juansmacbook/PycharmProjects/Santander_Transaction/santander_data/test.csv')
train=pd.read_csv('/Users/juansmacbook/PycharmProjects/Santander_Transaction/santander_data/train.csv')

"""reading data data"""

x_test=pd.read_csv('/Users/juansmacbook/PycharmProjects/Santander_Transaction/santander_data/x_test.csv')
x_train=pd.read_csv('/Users/juansmacbook/PycharmProjects/Santander_Transaction/santander_data/x_train.csv')

print('Data is ready!')


Loading data...
Data is ready!


In [15]:
x_train.columns[2:]


Index(['var_0', 'var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'var_6', 'var_7',
       'var_8', 'var_9',
       ...
       'var_190_unique', 'var_191_unique', 'var_192_unique', 'var_193_unique',
       'var_194_unique', 'var_195_unique', 'var_196_unique', 'var_197_unique',
       'var_198_unique', 'var_199_unique'],
      dtype='object', length=400)

Resampling

In [ ]:
training_set=x_train[x_train.columns[2:]].to_numpy()
training_labels=train['target'].to_numpy()

"""since we are dealing with an imbalanced data set we are going to deal with it  by oversampling the underrepresented class"""

from imblearn.over_sampling import RandomOverSampler
rov=RandomOverSampler()
x_resampled, y_resampled = rov.fit_resample(X=training_set, y=training_labels)

print('_.-._.-._.-._.-._.-._.-._.-._.-._.-._.-._.-._')
print('training set shape before resampling: '+str(training_set.shape))
print('training set shape after: '+str(x_resampled.shape))


print('_.-._.-._.-._.-._.-._.-._.-._.-._.-._.-._.-._')
print('training labels shape before resampling: '+str(training_labels.shape))
print('training labels shape after :'+str(y_resampled.shape))
print('_.-._.-._.-._.-._.-._.-._.-._.-._.-._.-._.-._')

In [ ]:



"""initializes a histogram gradient boost classifier """
def get_hgbc():
    hgbc=sklearn.ensemble.HistGradientBoostingClassifier(

        #parameters
        max_iter=1000,
        validation_fraction=0.02,
        max_depth=None,
        learning_rate=0.1,
        loss='binary_crossentropy',
        l2_regularization=0.01,
        scoring='roc_auc',
        max_leaf_nodes=40,
        verbose=1

    )
    return hgbc



"""Creates and fits model with the resampled data"""
model1=get_hgbc()
model1.fit(x_resampled, y_resampled, sample_weight=None)
print('Training done!')



"""Let's make predictions with the model"""
#real_samples=x_test[:100000] #real samples
#fake_samples=x_test[100000:] #fake samples
super_test_set=x_test[x_test.columns[2:]].to_numpy()
predictions=model1.predict(super_test_set)


#TEST
predictions_prob=model1.predict_proba(super_test_set)
predictions_prob=pd.DataFrame(predictions_prob)
######
predictions


"""Let's get ready for submission. Creating a DataFrame containing ID codes and targets"""
submission_df=pd.DataFrame()
submission_df['ID_code']=x_test['ID_code']

#TEST
#submission_df['target']=predictions THIS IS THE REAL DEAL
submission_df['target']=predictions_prob[1].values #TEST
########################

"""we are also creating a dummy columns to help us sort our data by ID code"""
submission_df['dummy_index']=submission_df['ID_code'].apply(lambda x: int(x[5:]))

submission_df

"""Let's sort our data and save it as a CSV file! We are done!"""
juan_submission=submission_df.sort_values(by='dummy_index')
juan_submission.drop(['dummy_index'],axis=1).to_csv('juan_submission_v2.csv', index=False)
final_submission=pd.read_csv(filepath_or_buffer='juan_submission_v2.csv')